<center>

<img src='https://micro.ce.sharif.edu/lib/tpl/writr/images/logo.svg' alt="SUT logo" width=500 height=300 align=center class="saturate" >


<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
Machine Learning <br>
Course Project, Fall 2025 <br>
<font color=3C99D size=5>
EEG - Motor Imagery Classification <br>
<font color=3C99D size=6>
By: Mohammad Hossein Hosseinzadeh 403203557 , Farshad Vaziri 403206179
Group:  G45

<br>
<br>
<br>

</center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import pi
import scipy.io as sio
from scipy.signal import butter, filtfilt
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, auc, silhouette_samples, silhouette_score, confusion_matrix
)



np.random.seed(0)


## Section 1.1: Data Loading and Windowing

### Overview

In this section, we load the EEG data from the BCI Competition IV dataset and perform temporal windowing. The continuous EEG signal is segmented into fixed-length windows based on motor imagery cues. The windowing process creates a standard machine learning dataset format where each sample corresponds to one motor imagery trial.

**Key Steps:**
1. Load continuous EEG signal and extract metadata (sampling frequency, channel names, marker positions)
2. Perform temporal windowing: extract 4-second segments starting from each cue marker
3. Create dataset in standard ML format (features + labels)
4. Display dataset statistics and visualize sample channels

**Dataset Characteristics:**
- **File:** BCICIV_calib_ds1a.mat
- **Channels:** 59 EEG electrodes
- **Sampling Frequency:** 1000 Hz
- **Classes:** Left hand (-1) and Foot (1)
- **Window Duration:** 4 seconds = 4000 samples

In [5]:
def load_and_window_data(mat_file, scale_factor=0.1):
    """
        windows: [trials, time, channels]
        labels:  [trials]
        fs:      sampling rate
        chNames: channel names
    """
    data = sio.loadmat(mat_file, squeeze_me=True, struct_as_record=False)

    cnt = scale_factor * data['cnt'].astype(float)  # [time x channels]
    mrk = data['mrk']
    nfo = data['nfo']

    fs = float(nfo.fs)
    chNames = list(nfo.clab)

    win_duration = 4.0  # seconds
    win_samples = int(round(win_duration * fs))

    cue_pos = mrk.pos
    cue_labels = mrk.y
    num_cues = len(cue_pos)

    num_channels = cnt.shape[1]
    windows = np.zeros((num_cues, win_samples, num_channels))
    labels = np.zeros(num_cues, dtype=int)

    for i in range(num_cues):
        start_idx = int(cue_pos[i])
        end_idx = start_idx + win_samples

        if end_idx > cnt.shape[0]:
            segment = cnt[start_idx:, :]
            padded = np.zeros((win_samples, num_channels))
            padded[:segment.shape[0], :] = segment
            windows[i, :, :] = padded
        else:
            windows[i, :, :] = cnt[start_idx:end_idx, :]

        labels[i] = int(cue_labels[i])

    return windows, labels, fs, chNames


### 1.1.1: Load Data and Display Statistics

The following code loads the raw EEG data from the .mat file using the `load_and_window_data()` function we defined earlier. This function:
- Extracts the continuous signal (cnt) and scales it
- Reads marker positions and labels from the cue information
- Performs windowing with a 4-second duration
- Returns windowed data in shape [num_trials, num_samples, num_channels]

We then print comprehensive statistics about the loaded dataset including dimensions, class distribution, and sampling information.

In [ ]:
# Load the data
windows, labels, fs, chNames = load_and_window_data("/mnt/c/VS/ML_PROJECT/data/BCICIV_1_mat/BCICIV_calib_ds1a.mat")

print(f"Total number of samples:        {windows.shape[0]}")
print(f"Window length (samples):        {windows.shape[1]}")
print(f"Window duration (seconds):      {windows.shape[1] / fs:.1f}")
print(f"Number of channels:             {windows.shape[2]}")
print(f"Sampling frequency (Hz):        {fs}")
print()
print(f"Unique classes:                 {np.unique(labels)}")
print(f"Class distribution:             {dict(zip(*np.unique(labels, return_counts=True)))}")



### 1.1.2: Visualize Sample Channels

Now we visualize one sample from the dataset to examine the raw EEG signals. We plot 5 specific channels (0, 15, 30, 45, 58) as time-series functions as required by the project specification. This visualization helps us understand the characteristics of the motor imagery signals before proceeding to preprocessing and feature extraction.

The plot shows:
- **X-axis:** Time in seconds (0 to 4 seconds)
- **Y-axis:** Signal amplitude in microvolts
- **5 Channels:** Distributed across the scalp (anterior, central, and posterior regions)
- **Signal Quality:** The raw EEG before any filtering

**Note on Train-Test Split:** Although we perform a 75/25 train-test split in the subsequent `run_pipeline()` function during classification (Section 2), we display statistics and visualizations on the full dataset here to understand its overall characteristics. The actual train-test split happens later when we process the filtered data and extract features for model training.

In [ ]:
# Visualize one sample - channels 0, 15, 30, 45, 59
sample_idx = 0
channels_to_plot = [0, 15, 30, 45, 58]

time_axis = np.arange(windows.shape[1]) / fs

fig, axes = plt.subplots(len(channels_to_plot), 1, figsize=(14, 10))

for i, ch_idx in enumerate(channels_to_plot):
    axes[i].plot(time_axis, windows[sample_idx, :, ch_idx], linewidth=1.5, color='steelblue')
    axes[i].set_ylabel(f'Ch {ch_idx}\n({chNames[ch_idx]})', fontsize=10)
    axes[i].grid(True, alpha=0.3)
    axes[i].set_xlim([0, time_axis[-1]])

axes[-1].set_xlabel('Time (seconds)', fontsize=12)
fig.suptitle(f'Sample #{sample_idx} - Motor Imagery EEG Signal', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 1.1.3: Train-Test Split (75/25)

According to the project specification, after loading and windowing the data, we must split it into training (75%) and testing (25%) sets. This split is performed on the raw windowed data before any filtering or feature extraction. This ensures that the train and test sets represent independent data splits, which is essential for unbiased model evaluation.

In [ ]:
#  Perform Train-Test Split (75/25) on raw windowed data
num_samples = windows.shape[0]
idx = np.random.permutation(num_samples)
train_count = int(round(0.75 * num_samples))
train_idx = idx[:train_count]
test_idx = idx[train_count:]

X_train_raw = windows[train_idx]
y_train = labels[train_idx]
X_test_raw = windows[test_idx]
y_test = labels[test_idx]


print(f"Total samples:                  {num_samples}")
print(f"Training samples (75%):         {len(X_train_raw)}")
print(f"Test samples (25%):             {len(X_test_raw)}")
print(f"Training set shape:             {X_train_raw.shape}")
print(f"Test set shape:                 {X_test_raw.shape}")
print(f"Training class distribution:    {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Test class distribution:        {dict(zip(*np.unique(y_test, return_counts=True)))}")
